In [0]:
import sys

sys.path.append("../include_utils/")

import ipyparallel as ipp
import os, time
import include_utils as u
import pandas as pd
import numpy as np
import scipy as sp
import numbers
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
import matplotlib.cm as cm
import matplotlib.colors as mcolors
import cyvcf
import vcf
from sklearn import preprocessing
from subprocess import Popen, PIPE
import seaborn as sns
from IPython.display import FileLink
import urllib2
import dill
import traceback
from pandas import Series, DataFrame
import gzip
import warnings
warnings.filterwarnings('ignore',category=pd.io.pytables.PerformanceWarning)
%config InlineBackend.figure_format = 'retina'
from Bio import SeqIO
import pysam
from collections import OrderedDict, namedtuple
import operator
import multiprocessing as mp
from hdfstorehelper import HDFStoreHelper
import dill

samtools = "/home/cfriedline/gpfs/src/samtools-1.2/samtools"
bcftools = "/home/cfriedline/gpfs/src/bcftools-1.2/bcftools"
picard = "/home/cfriedline/gpfs/src/broadinstitute-picard-03a1d72/dist/picard.jar"
java = "/home/cfriedline/g/src/jdk1.8.0_60/bin/java"
perl = "/home/cfriedline/gpfs/opt/ActivePerl-5.16/bin/perl"

vcfutils = "perl /home/cfriedline/g/src/bcftools-1.2/vcfutils.pl"
vcftools = "/home/cfriedline/bin/vcftools"
bcftools = "/home/cfriedline/gpfs/src/bcftools-1.2/bcftools"
tabix = "/home/cfriedline/gpfs/src/samtools-1.2/htslib-1.2.1/tabix"
bgzip = "/home/cfriedline/gpfs/src/samtools-1.2/htslib-1.2.1/bgzip"


def setup_r():
    os.environ['R_HOME'] = '/home/cfriedline/g/R3/lib64/R'
    os.environ['LD_LIBRARY_PATH'] = "%s/lib:%s:%s" % (os.environ['R_HOME'], 
                                                   os.environ['LD_LIBRARY_PATH'],
                                                     "/home/cfriedline/lib64")

setup_r()
import rpy2.robjects as robjects
from rpy2.robjects import pandas2ri
pandas2ri.activate()
r = robjects.r

%reload_ext autoreload
%autoreload 2
%matplotlib inline
%reload_ext rpy2.ipython

In [0]:
notimputed_vcf = "/home/cfriedline/eckertlab/gypsy_indiv/masked/analysis/samtools1.2_no_otis/notimputed/isect.vcf"
imputed_vcf = '/home/cfriedline/eckertlab/gypsy_indiv/masked/analysis/samtools1.2_no_otis/beagle40/isect.vcf'
notimputed_vcf_gz = "%s.gz" % notimputed_vcf
imputed_vcf_gz = "%s.gz" % imputed_vcf

In [0]:
hdf_files = [os.path.join(os.path.dirname(x), "isect.hd5") for x in [notimputed_vcf_gz, imputed_vcf_gz]]

In [0]:
hdfs = [HDFStoreHelper(x) for x in hdf_files]

In [0]:
hdfs[0].get_group_names()

In [0]:
hierf_trans = {0:11, 1:12, 2:22, -1:'NA'}
def apply_hierf_trans(series):
    return [hierf_trans[x] if x in hierf_trans else x for x in series]

In [0]:
z12_swapped = [x.get('z12_swapped') for x in hdfs]

In [0]:
hierf_df = [x.ix[:,:-2].apply(apply_hierf_trans) for x in z12_swapped]

In [0]:
for i, df in enumerate(hierf_df):
    hierf_df[i] = z12_swapped[i][['popid']].join(df)

In [0]:
for i, elem in enumerate(hdf_files):
    filedir = os.path.dirname(elem)
    outfile = os.path.join(filedir, "isect_hierfstat.txt")
    hierf_df[i].to_csv(outfile, header=True, index=False, sep="\t")

### Put into R (because it can be slow)

```R
library(hierfstat)
data = read.table("isect_hierfstat.txt", header=T, sep="\t")
levels = data.frame(data$popid)
loci = data[,2:ncol(data)]
bs = basic.stats(data)
saveRDS(bs, "isect_hierfstat_basic_stats.rds")
res = varcomp.glob(levels=levels, loci=loci, diploid=T)
saveRDS(res, "isect_hierfstat_varcomp.rds")

```